In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, f1_score, classification_report
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive')

Mounted at /content/drive


Carregando dados de treino e teste

In [ ]:
X_train = np.load('/content/drive/MyDrive/outfile_X_train.npy', allow_pickle=True)
X_test = np.load('/content/drive/MyDrive/outfile_X_test.npy', allow_pickle=True)
y_train = np.load('/content/drive/MyDrive/outfile_y_train.npy', allow_pickle=True)
y_test = np.load('/content/drive/MyDrive/outfile_y_test.npy', allow_pickle=True)

Função que remove elementos vazios da lista

In [ ]:
def remover(X,y):
     aux = []
     ret = 0
     x_aux = []
     y_aux = []
     for i in range(len(y)):
         if y[i] != []:
           y_aux.append(y[i])
           x_aux.append(X[i])
     return np.array(x_aux), np.array(y_aux)

In [ ]:
X_train,y_train = remover(X_train,y_train)
X_test,y_test = remover(X_test,y_test)

<ipython-input-4-c44d1cdeb708>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(x_aux), np.array(y_aux)


In [ ]:
print("X_train:" + str(X_train.shape) + "; y_train:" + str(y_train.shape) + "; X_test:" + str(X_test.shape) + "; y_test:" + str(y_test.shape))

X_train:(19237, 1000, 12); y_train:(19237,); X_test:(2159, 1000, 12); y_test:(2159,)


Função que realiza codificação multi-hot

In [ ]:
def multi_hot_encoding(y):
    #Obter dados categóricos
    y_encoded = np.array(y)

    # Criar uma instância do codificador multi-hot
    encoder = MultiLabelBinarizer()

    # Aplicar a codificação multi-hot aos dados
    y_encoded_ = encoder.fit_transform(y_encoded)

    return y_encoded_

In [ ]:
y_train = multi_hot_encoding(y_train)
y_test = multi_hot_encoding(y_test)

In [ ]:
y_train

array([[0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0]])

In [ ]:
y_train.shape

(19237, 5)

In [ ]:
y_test

array([[0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0]])

In [ ]:
y_test.shape

(2159, 5)

Arquitetura Transformer

In [ ]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, input_dim, n_heads, ff_dim, dropout=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.attention = layers.MultiHeadAttention(
            num_heads=n_heads, key_dim=input_dim[-1] // n_heads
        )
        self.ff = tf.keras.Sequential([
            layers.Dense(ff_dim, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(dropout),
        ])

    def build(self, input_shape):
        #Adiciona a camada densa residual
        self.dense_residual = layers.Dense(input_shape[-1])

    def call(self, inputs):
        attention_output = self.attention(inputs, inputs)
        x = tf.keras.layers.Add()([attention_output, inputs])  #Conexão residual

        #Ajuste das dimensões usando Dense
        residual = self.dense_residual(x)
        x = tf.keras.layers.Add()([x, residual])  #Conexão residual

        x = self.ff(x)
        return x


# Definindo o modelo Transformer
class Transformer(keras.Model):
    def __init__(self, n_blocks, n_heads, ff_dim, n_classes, input_shape, rate=0.1):
        super(Transformer, self).__init__()
        self.encoder_layers = [TransformerBlock(input_shape, n_heads, ff_dim, rate) for _ in range(n_blocks)]
        self.flatten = layers.Flatten()
        self.f_connected = layers.Dense(n_classes, activation="sigmoid")

    def call(self, inputs):
        x = inputs
        for layer in self.encoder_layers:
            x = layer(x)
        x = self.flatten(x)
        x = self.f_connected(x)
        return x

In [ ]:
input_dim = X_train.shape[1:]
n_blocks = 6
n_heads = 8
ff_dim = 512
n_classes = 5

#Criar instância do modelo Transformer
model = Transformer(n_blocks=n_blocks, n_heads=n_heads, ff_dim=ff_dim, n_classes=n_classes, input_shape=input_dim, rate=0.1)

#Criar um tensor de entrada baseado nas dimensões desejadas
input_dim_ = tf.keras.Input(shape=input_dim)
model(input_dim_)  # Chamando o modelo com o tensor de entrada

#Compilar o modelo depois de chamar o modelo com o tensor de entrada
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss="binary_crossentropy",
              metrics=["accuracy",
                        tf.keras.metrics.Precision(),
                        tf.keras.metrics.Recall(),
              ])
model.summary()


Model: "transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer_block (Transfo  multiple                  9280      
 rmerBlock)                                                      
                                                                 
 transformer_block_1 (Trans  multiple                  544280    
 formerBlock)                                                    
                                                                 
 transformer_block_2 (Trans  multiple                  544280    
 formerBlock)                                                    
                                                                 
 transformer_block_3 (Trans  multiple                  544280    
 formerBlock)                                                    
                                                                 
 transformer_block_4 (Trans  multiple                  

Realizando treinamento do modelo

In [ ]:
epochs = 30
batch_size = 32
early_stopping = EarlyStopping(patience=10, min_delta=1e-3, monitor="val_accuracy", restore_best_weights=True)


history = model.fit(X_train,
                    y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split=0.1,
                    callbacks=[early_stopping]
)

Epoch 1/30
542/542 [==============================] - 467s 861ms/step - loss: 0.5619 - accuracy: 0.4006 - precision: 0.4472 - recall: 0.1495 - val_loss: 0.6216 - val_accuracy: 0.3378 - val_precision: 0.3504 - val_recall: 0.2617
Epoch 2/30
542/542 [==============================] - 467s 861ms/step - loss: 0.5363 - accuracy: 0.4344 - precision: 0.5040 - recall: 0.1848 - val_loss: 0.6263 - val_accuracy: 0.3373 - val_precision: 0.3768 - val_recall: 0.2719
Epoch 3/30
542/542 [==============================] - 466s 860ms/step - loss: 0.5387 - accuracy: 0.4206 - precision: 0.5006 - recall: 0.1991 - val_loss: 0.6596 - val_accuracy: 0.3347 - val_precision: 0.3456 - val_recall: 0.2598
Epoch 4/30
542/542 [==============================] - 466s 860ms/step - loss: 0.5389 - accuracy: 0.4255 - precision: 0.5079 - recall: 0.2347 - val_loss: 0.6172 - val_accuracy: 0.3488 - val_precision: 0.4090 - val_recall: 0.2055
Epoch 5/30
542/542 [==============================] - 466s 860ms/step - loss: 0.5493 - a

Salvar o melhor modelo com base em val_accuracy

In [ ]:
model.save("transformer_3_", save_format="tf")

Análise do modelo

In [ ]:
history.history

{'loss': [0.5619253516197205,
  0.5363001227378845,
  0.5387309789657593,
  0.538870632648468,
  0.549344003200531,
  0.5551968812942505,
  4.588216781616211,
  0.8498784303665161,
  0.558519184589386,
  0.5665244460105896,
  0.5784583687782288,
  0.5978090763092041,
  0.6100758910179138,
  5.7399163246154785,
  0.5602045059204102],
 'accuracy': [0.400566041469574,
  0.4343556761741638,
  0.420608788728714,
  0.4254606366157532,
  0.4136775732040405,
  0.3987177312374115,
  0.3534916043281555,
  0.3610004186630249,
  0.3996996581554413,
  0.3824293911457062,
  0.35926759243011475,
  0.34101542830467224,
  0.3269219696521759,
  0.34944838285446167,
  0.4231502413749695],
 'precision': [0.4471718370914459,
  0.5040135979652405,
  0.5006167888641357,
  0.5078645348548889,
  0.5113646388053894,
  0.4880732297897339,
  0.395484060049057,
  0.3981633186340332,
  0.4502483010292053,
  0.4249100685119629,
  0.43396952748298645,
  0.40689364075660706,
  0.3832519054412842,
  0.34599658846855164

In [ ]:
evaluation_results = model.evaluate(X_test, y_test)

68/68 [==============================] - 21s 313ms/step - loss: 0.6304 - accuracy: 0.4317 - precision: 0.4741 - recall: 0.3565


In [ ]:
y_pred = model.predict(X_test, verbose=False)

In [ ]:
y_pred

array([[0.25941467, 0.02733217, 0.30563647, 0.7355183 , 0.05196291],
       [0.22712983, 0.01646767, 0.18642433, 0.8651503 , 0.06393582],
       [0.28283232, 0.02652532, 0.3126681 , 0.6858861 , 0.06823549],
       ...,
       [0.12253928, 0.01229578, 0.1389898 , 0.91863394, 0.03708415],
       [0.23386201, 0.02653564, 0.3294926 , 0.73596144, 0.06094231],
       [0.33113045, 0.036885  , 0.4814163 , 0.4830795 , 0.08248489]],
      dtype=float32)

In [ ]:
y_test

array([[0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0]])

In [ ]:
#define limiar(threshold) para converter as previsões contínuas em rótulos binários e poder usar o classification_report
threshold = 0.3

#aplicando o limiar às previsões
y_pred_ = (y_pred > threshold).astype(int)

print(classification_report(y_test, y_pred_))

              precision    recall  f1-score   support

           0       0.37      0.27      0.31       497
           1       0.00      0.00      0.00       262
           2       0.40      0.53      0.45       551
           3       0.45      1.00      0.62       963
           4       0.33      0.00      0.00       521

   micro avg       0.43      0.50      0.46      2794
   macro avg       0.31      0.36      0.28      2794
weighted avg       0.36      0.50      0.36      2794
 samples avg       0.47      0.57      0.49      2794



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
accuracy = accuracy_score(y_test,y_pred_)
print(accuracy)

0.32468735525706344


In [ ]:
#define limiar(threshold) para converter as previsões contínuas em rótulos binários e poder usar o classification_report
threshold = 0.5

#aplicando o limiar às previsões
y_pred_ = (y_pred > threshold).astype(int)

print(classification_report(y_test, y_pred_))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       497
           1       0.00      0.00      0.00       262
           2       0.70      0.08      0.15       551
           3       0.47      0.99      0.63       963
           4       1.00      0.00      0.00       521

   micro avg       0.47      0.36      0.41      2794
   macro avg       0.43      0.21      0.16      2794
weighted avg       0.48      0.36      0.25      2794
 samples avg       0.46      0.44      0.45      2794



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
accuracy = accuracy_score(y_test,y_pred_)
print(accuracy)

0.4242704955998147


In [ ]:
#define limiar(threshold) para converter as previsões contínuas em rótulos binários e poder usar o classification_report
threshold = 0.75

#aplicando o limiar às previsões
y_pred_ = (y_pred > threshold).astype(int)

print(classification_report(y_test, y_pred_))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       497
           1       0.00      0.00      0.00       262
           2       0.00      0.00      0.00       551
           3       0.54      0.76      0.63       963
           4       1.00      0.00      0.00       521

   micro avg       0.54      0.26      0.35      2794
   macro avg       0.31      0.15      0.13      2794
weighted avg       0.37      0.26      0.22      2794
 samples avg       0.34      0.33      0.33      2794



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
accuracy = accuracy_score(y_test,y_pred_)
print(accuracy)

0.3219082908754053


In [ ]:
print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       497
           1       0.00      0.00      0.00       187
           2       0.27      0.09      0.13       317
           3       0.44      0.99      0.61       916
           4       0.00      0.00      0.00       242

    accuracy                           0.43      2159
   macro avg       0.14      0.21      0.15      2159
weighted avg       0.23      0.43      0.28      2159



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
accuracy = accuracy_score(y_test,y_pred_)
print(accuracy)

0.3219082908754053
